In [ ]:
import pandas as pd
import numpy as np
import math

# Finance
import mplfinance as mpf
import ta
import yfinance as yf

import torch
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from torch.optim import Adam

import matplotlib.pyplot as plt
import shap



# System
from dotenv import load_dotenv
import os
from pathlib import Path
import requests
import sys
import time

sys.path.append('../') # Change the python path at runtime
from src.utils import path as path_yq


In [ ]:
load_dotenv()
cur_dir = Path.cwd()

POLYGON_API_KEY = os.environ.get("POLYGON_API_KEY")

# Fetch Tick Data
- Add feature to pull from data instead of fetching

## Polygon API

In [ ]:
# TODO: Scale this up to n years
ticker = "NVDA"
max_limit = 50000
start_date = "2000-01-01"
end_date = "2024-12-31"

api_url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/{start_date}/{end_date}?adjusted=true&sort=asc&limit={max_limit}&apiKey={POLYGON_API_KEY}"



In [ ]:
str = "N" # Reset
str = input("Confirm?")

if str == "Y":
    resp = requests.get(api_url)
    print(f"Request made.")

In [ ]:
resp

In [ ]:
cols = ["Date", "Open", "High", "Low", "Close", "Volume", "VWAP", "Transactions"]
if resp.status_code == 200:
    dict_list = resp.json().get('results')
    df = pd.DataFrame(dict_list)

    column_map = {
        't': 'Timestamp',
        'o': 'Open',
        'h': 'High',
        'l': 'Low',
        'c': 'Close', 
        'n': 'Transactions', # Number of trades (market activity)
        'v': 'Volume', # Number of shares traded (intensity of the activity)
        'vw': 'VWAP'
    }

    df.rename(columns=column_map, inplace=True)

    df['Datetime'] = pd.to_datetime(df['Timestamp'], unit='ms')
    df['Date'] = df['Datetime'].dt.normalize() # Remove the time and return date object

    df = df[cols]
    df.set_index(keys="Date", inplace=True)
else:
    print(f"Error fetching data: {resp.status_code}, {resp.text}")

In [ ]:
df

## Yfinance

In [ ]:
import seaborn as sns
historical_start_date = '2022-08-09'
# Define the ticker list
# Capitaland A17U, SUNT.SI cannot download
ticker_list = ['S51.SI']

# Fetch the data
data = yf.download(ticker_list, historical_start_date)['Adj Close'] # Auto adjust is false
data.index = pd.to_datetime(data.index)
display(data.tail(20))
data.plot()

plt.figure(figsize=(3,2))
sns.heatmap(data.corr(), cmap="Reds", annot=True)
plt.show()

In [ ]:
df


## Read Data

In [ ]:
root_dir = path_yq.get_root_dir(cur_dir=cur_dir)
csv_path = Path.joinpath(root_dir, "data", f"{ticker}_{start_date}_{end_date}.csv")

# Get df
try:
    df.to_csv(csv_path)
except NameError:
    print(f"df not defined, trying to fetch from csv")
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        # Convert data
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index(keys="Date", inplace=True)

In [ ]:
df.tail(6)

mpf plot: https://github.com/matplotlib/mplfinance?tab=readme-ov-file

In [ ]:
mpf.plot(df, type='candle', style='charles', figsize=(20, 10), title="OHLC Bars for NVDA", volume=True, show_nontrading=True, mav=(3, 6, 9))

In [ ]:
df.isna().sum()


## Read and Preprocess Bryan's Data


In [ ]:
csv_path = Path.joinpath(path_yq.get_root_dir(cur_dir=cur_dir), 'data', 'capitaland.csv')
df2 = pd.read_csv(csv_path)
df2['Date'] = pd.to_datetime(df2['Date']).dt.normalize()

assert (df2.index == df2.sort_values(by='Date', ascending=True).index).all(), 'df is not sorted in ascending'
df2['Year'] = df2.Date.dt.year
df2['Month'] = df2.Date.dt.month
df2['Day'] = df2.Date.dt.day

if input('Add additional features?') == 'Y':
    # Testing additional features
    df2['BB High'], df2['BB Low']= ta.volatility.bollinger_hband(df2['A17U.SI_Close']), ta.volatility.bollinger_lband(df2['A17U.SI_Close'])

    df2['MA_50'] = df2['A17U.SI_Close'].rolling(window=50).mean()

    df2['MA_20'] = df2['A17U.SI_Close'].rolling(window=20).mean()

    df2['MA_5'] = df2['A17U.SI_Close'].rolling(window=5).mean()
df2.drop(columns='Date', inplace=True)
df2.dropna(inplace=True)
print(df2.isna().sum())

df2

# Features
- Technical indicators
- Fundamental indicators
- Date features
- Holiday indicators etc.

# Preprocessing
- Train-test split
- Scale data (only fit-transform for train, but not for test)
- Decide which to predict. Have open and predict the close for the same day? Have the close for the previous day and predict next open?

In [ ]:
def create_technical_indicators(tmp: pd.DataFrame):
    df = tmp.copy(deep=True)

    # Date features
    # df['Year'], df['Month'], df['Day'] = df.index.year, df.index.month, df.index.day
    # FIXME: Test no date
    # Use the close price to create the indicators
    df['BB High'], df['BB Low']= ta.volatility.bollinger_hband(df['Close']), ta.volatility.bollinger_lband(df['Close'])
    
    # Takes the past 20 data, including the current one to calculate the mean
    df['MA_50'] = df['Close'].rolling(window=50).mean()

    df['MA_20'] = df['Close'].rolling(window=20).mean()

    df['MA_5'] = df['Close'].rolling(window=5).mean()

    df['Future Close'] = df['Close'].shift(-1)

    return df

In [ ]:
df2 = create_technical_indicators(tmp=df)

In [ ]:
df2

In [ ]:
df2.dropna(inplace=True)
# Removes the row that doesn't have a target also
df2

## Scale, Load Dataset

In [ ]:
start_time = time.time()
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time is {elapsed_time:3f}s.")

In [ ]:
class Scaler:
    def __init__(self, type='standard') -> None:
        self.scaler = None
        if type == 'standard':
            self.scaler = StandardScaler()
        elif type == 'minmax':
            self.scaler = MinMaxScaler()
        
    def scale_fit_transform(self, train, val):
        train_scaled = self.scaler.fit_transform(train)
        val_scaled = self.scaler.transform(val)

        return train_scaled, val_scaled

    def scale_transform(self, test):
        return self.scaler.transform(test)


# def scale(X_train, X_val, type='standard'):
#     """
#     Scale based on the train and val/test data with a specific type.
#     """
#     type = 'standard'
#     if type == 'standard':
#         scaler = StandardScaler()
#     else:
#         scaler = MinMaxScaler()

#     X_train_scaled = scaler.fit_transform(X_train)
#     X_val_scaled = scaler.transform(X_val)

#     return X_train_scaled, X_val_scaled

In [ ]:
def convert_numpy_torch(arr) -> torch:
    if isinstance(arr, (pd.DataFrame, pd.Series)):
        arr = arr.to_numpy()
    if isinstance(arr, np.ndarray):
        # This does not make a copy, but any changes will affect the original arr
        # Alternative: torch.tensor(arr, dtype=torch.float32)
        return torch.from_numpy(arr).float().to(DEVICE) # Ensure it is a float tensor, and move to device
    else:
        raise ValueError(f"The input\n{arr}\nis not an ndarray, it is a {type(arr)}.")

# Only X needs to be turned into a sequence
def load_sequence(X, seq_len):
    sequences = []
    for i in range(len(X) - seq_len):
        sequences.append(X[i: i + seq_len])
    print(np.array(sequences).shape)
    return np.array(sequences)

def load_data(X, y, batch_size, shuffle):
    dataset = TensorDataset(convert_numpy_torch(X),
                                convert_numpy_torch(y))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader


# Model Class

In [ ]:
class EarlyStopper:
    def __init__(self, patience=5):
        self.min_loss = np.inf
        self.patience = patience
        self.counter = 0
        self.delta = 0 # TODO: Might wanna adjust based on stock volatility

    def stop(self, loss: float) -> bool:
        if loss < self.min_loss - self.delta:
            self.min_loss = loss
            # Reset counter
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False



LSTM self-define ReLU: https://discuss.pytorch.org/t/change-tanh-activation-in-lstm-to-relu/14160

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size=None, num_layers=None, **kwargs):
        super().__init__() # Need bracket for super
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = 1
        self.bidir = False # Hardcoded since bidir won't be used for stock price pred

        self.rnn = nn.RNN(
            input_size=input_size, 
            hidden_size=self.hidden_size, 
            num_layers=self.num_layers, # Dropout has no effect on single hidden layer
            nonlinearity='tanh', # or relu
            bias=True,
            batch_first=False,
            dropout=0.2,
            bidirectional=self.bidir, # Can be True for NLP, but will introduce lookahead bias for stock
        )

        total_features = self.hidden_size * 2 if self.bidir else self.hidden_size
        # Use linear layer for fully connected layer to map to 1 column of output
        self.fc = nn.Linear(
            in_features=total_features,
            out_features=self.output_size
            )
    
    # Must override the parent class's forward method
    def forward(self, x):
        rnn_out, _ = self.rnn(x)

        # print(rnn_out.shape) # [batch_size, features]

        return self.fc(rnn_out)

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size=None, num_layers=None, **kwargs):
        super().__init__() # Need bracket for super
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = 1
        self.bidir = False # Hardcoded since bidir won't be used for stock price pred

        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=self.hidden_size, 
            num_layers=self.num_layers, # Dropout has no effect on single hidden layer
            bias=True,
            batch_first=False,
            dropout=0.2,
            bidirectional=self.bidir, # Can be True for NLP, but will introduce lookahead bias for stock
        )

        total_features = self.hidden_size * 2 if self.bidir else self.hidden_size
        # Use linear layer for fully connected layer to map to 1 column of output
        self.fc = nn.Linear(
            in_features=total_features,
            out_features=self.output_size
            )
    
    # Must override the parent class's forward method
    def forward(self, x):
        lstm_out, _ = self.lstm(x)

        # print(rnn_out.shape) # [batch_size, features]
        return self.fc(lstm_out)
        

In [ ]:
from collections import defaultdict
from sklearn.metrics import root_mean_squared_error

class custom_model():
    def __init__(self, type='RNN', lr=None, n_epoch=None, device=None, **kwargs):
        self.lr = lr
        self.n_epoch = n_epoch
        self.input_size = kwargs.get('input_size', None)
        self.hidden_size = kwargs.get('hidden_size', 1)
        self.num_layers = kwargs.get('num_layers', 1)
        self.train_dataloader = kwargs.get('train_dataloader', None)
        self.val_dataloader = kwargs.get('val_dataloader', None)
        self.scaler_y = kwargs.get('scaler_y', None)
        self.device = device
        
        if type == 'RNN':
            self.model = RNN(
                input_size=self.input_size,
                hidden_size=self.hidden_size,
                num_layers=self.num_layers
            ).to(device)
        elif type == 'LSTM':
            self.model = LSTM(
                input_size=self.input_size,
                hidden_size=self.hidden_size,
                num_layers=self.num_layers
            ).to(device)
        self.optimiser = Adam(params=self.model.parameters(), lr=lr)
        self.loss_fn = nn.MSELoss() # Use sqrt for RMSE
        self.model_dict = defaultdict(list)
        self.early_stopper = EarlyStopper()

    def fit(self):
        for epoch in range(self.n_epoch):
            total_train_loss = 0
            total_val_loss = 0
            total_train_samples = 0
            total_val_samples = 0
            pred_dict = defaultdict(list)
            
            start_time = time.time()
            
            self.model.train()
            for input, target in self.train_dataloader:
                # input, target = input.to(self.device), target.to(self.device)
                self.optimiser.zero_grad() # Resets gradient of the optimised Tensors to None
                output = self.model(input)
                # print(output.shape, target.shape)
                output = output.squeeze(-1)
                # assert output.squeeze(-1).shape == target.shape
                # Using RMSE for both train and eval
                
                loss = self.loss_fn(output, target)
                # print(input.shape[0])
                # print(loss.item())
                total_train_loss += loss.item() * input.shape[0] # Tensor operation: get the scalar in a tensor with 1 element
                total_train_samples += input.shape[0]

                loss.backward() # Compute the gradient of the loss wrt weights, backpropagate

                # for name, param in self.model.named_parameters():
                #     if param.grad is not None:
                #         print(f"Layer: {name} | Max Gradient: {param.grad.data.abs().max()}")
                self.optimiser.step() # Takes a step in the direction that reduces the loss, updates params
            
            self.model.eval() # Disable dropout
            with torch.no_grad():
                for input, target in self.val_dataloader:
                    # print(f"Before to device: Input: {input.device}, target: {target.device}")
                    # input, target = input.to(self.device), target.to(self.device) 
                    # Manually move to device even though dataloader ady moved
                    # Doesn't really make a difference
                    output = self.model(input)
                    output = output.squeeze(-1)
                    loss = self.loss_fn(output, target)
                    total_val_loss += loss.item() * input.shape[0]
                    total_val_samples += input.shape[0]

                    # Not efficient since it's only used after all epochs end
                    pred_dict['predicted'].append(self.scaler_y.scaler.inverse_transform(np.reshape(output.tolist(), (-1, 1)))) # Convert tensor to list
                    pred_dict['actual'].append(self.scaler_y.scaler.inverse_transform(np.reshape(target.tolist(), (-1, 1))))
            
            # Calculating and logging metadata
            # Add train, val loss to dict for each epoch
            avg_train_loss = math.sqrt(total_train_loss / total_train_samples) # Take the mean of MSE for all batches 
            avg_val_loss = math.sqrt(total_val_loss / total_val_samples)

            self.model_dict['train_loss'].append(avg_train_loss)
            self.model_dict['val_loss'].append(avg_val_loss)
            
            end_time = time.time()
            elapsed_time = end_time - start_time
            self.model_dict['time'].append(elapsed_time)
            # print(f"Elapsed time is {elapsed_time:3f}s.")

            if self.early_stopper.stop(loss=avg_val_loss) == True:
                print(f"Early stopping at epoch {epoch + 1}.")
                break

            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1} has completed.")

        # Only plot the predicted vs actual for the last epoch
        pred_dict['predicted'] = [item for sublist in pred_dict['predicted'] for item in sublist]
        pred_dict['actual'] = [item for sublist in pred_dict['actual'] for item in sublist]

        self.plot_pred_actl(actual_list=pred_dict['actual'], pred_list=pred_dict['predicted'])

        # After the whole training is completed, we can plot the losses, time and analyse which model is the best 
        # Train with full set and save the model for testing

    def evaluate(self, test_dataloader):
        """
        Predict the test dataset that has labels but not used in training phase.
        """

        total_test_loss = 0
        total_test_samples = 0
        self.model.eval() # Disable dropout

        pred_dict = defaultdict(list)
        with torch.no_grad():
            for input, target in test_dataloader:
                output = self.model(input)
                output = output.squeeze(-1)
                loss = self.loss_fn(output, target)
                total_test_loss += loss.item() * input.shape[0]
                total_test_samples += input.shape[0]

                pred_dict['predicted'].append(self.scaler_y.scaler.inverse_transform(np.reshape(output.tolist(), (-1, 1)))) # Convert tensor to list
                pred_dict['actual'].append(self.scaler_y.scaler.inverse_transform(np.reshape(target.tolist(), (-1, 1))))

        pred_dict['predicted'] = [item for sublist in pred_dict['predicted'] for item in sublist]
        pred_dict['actual'] = [item for sublist in pred_dict['actual'] for item in sublist]
        avg_test_loss = math.sqrt(total_test_loss / total_test_samples)

        # pred_list = pred_dict['predicted']
        # actual_list = pred_dict['actual']

        # rmse = root_mean_squared_error(y_true=actual_list, y_pred=pred_list)
        # print(f"RMSE: {rmse:.5f}, average test loss: {avg_test_loss:.5f} should be similar") 

        self.plot_pred_actl(pred_list=pred_dict['predicted'], actual_list=pred_dict['actual'])
        
        return pred_dict, avg_test_loss

    def predict(self):
        """
        Predicts unforeseen data without any evaluation.
        """

    def plot_pred_actl(self, pred_list, actual_list):
        plt.figure(figsize=(10, 6))
        x = range(len(pred_list))

        plt.plot(x, pred_list, label='Predicted')
        plt.plot(x, actual_list, label='Actual')
        plt.title('Actual vs Predicted') # TODO: Add in val or test
        plt.legend()
        plt.show()
    
    def plot_loss(self):
        model_df = pd.DataFrame(self.model_dict)
        plt.figure(figsize=(5, 3))
        plt.plot(np.arange(1, len(model_df) + 1), model_df[['train_loss']], label='Train Loss')
        plt.plot(np.arange(1, len(model_df) + 1), model_df[['val_loss']], label='Validation Loss')
        final_train_loss = model_df[['train_loss']].iloc[-1].item()
        final_val_loss = model_df[['val_loss']].iloc[-1].item()
        plt.axhline(final_train_loss, color='g', linestyle='--', label=f"{final_train_loss:.2f}")
        plt.axhline(final_val_loss, color='r', linestyle='--', label=f"{final_val_loss:.2f}")
        plt.title(f"Train vs Validation Loss")
        plt.xlabel(f"Epoch")
        plt.ylabel(f"RMSE Loss")
        plt.legend()
        plt.show()
        

# Split data - testing

- Need to have train, val, test
- General requirements:
    - Ideally no NA values, otherwise model performance might be affected
    - Number of rows TBC, cannot be too little (not enough data to train), and cannot 
    be too much also (computationally expensive)
- Specific requirements:
    - df2 (not sure if need date as index yet)
    - Features (should be able to have both categorical and numerical)
    - Target variable: Future Close
    - df2 can only contain numerical variables for now, if there are datetime objects
    or other types, need to convert to categorical (ColumnTransformer, Pipeline, OneHotEncoder()) and numerical.

In [ ]:
# We still need the current Close to predict the next close
cfm = 'N'
cfm = input('Run Bryan data?')
if cfm == 'Y':
    target_col_name = 'A17U.SI_Next_Close'
    X = df2.drop(columns=[target_col_name])
    y = df2[target_col_name]
else:
    X = df2.drop(columns=['Future Close'])
    y = df2['Future Close']

In [ ]:
X

In [ ]:
# iterdata = iter(train_dataloader)
# input, label = next(iterdata)
# input.shape
# print(len(train_dataloader), input.shape)
# # This is len_seq, batch_size, n_features

Using different training size to perform time series split/expanding window can
help understand how the model performs with different subsets of data.

If the model performs worse with a larger set of data, the smaller dataset might
not be able to generalise the overall trend, which indicates a potential shift
in market trends. Additionally, the features are not likely to have a high predictive
power on the target.

In [ ]:
DEVICE = torch.device('mps')
device = torch.device('mps') # torch.device('cpu')
n_splits = 3
split_idx = int(len(df2) * 0.8)
scaler_type = 'minmax' # Standard scaler does not provide good results
batch_size = 1 # Larger batch size takes longer to converge, tried 64 and 32
shuffle = False

# FIXME: Assess the model on different sets and use the best one
# TODO: Gap
tscv = TimeSeriesSplit(n_splits=n_splits, gap=10)

# TODO: Assess the models
# Now do CV for assessing robustness, but later on, the train and validation should
# Choose the best model/just go according to the sequential split
print(f"Length train + val, test: {split_idx}, {len(df2) - split_idx}")
for train_idx, val_idx in tscv.split(X.iloc[:split_idx]):
    print(f"Length train, val: {len(train_idx)}, {len(val_idx)}.")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    scaler_x, scaler_y = Scaler(type=scaler_type), Scaler(type=scaler_type)
    X_train_scaled, X_val_scaled = scaler_x.scale_fit_transform(train=X_train, val=X_val)
    y_train_scaled, y_val_scaled = scaler_y.scale_fit_transform(
        train=np.reshape(y_train, (-1, 1)), 
        val=np.reshape(y_val, (-1, 1)))
    
    train_dataloader = load_data(X=X_train_scaled, y=y_train_scaled, batch_size=batch_size, shuffle=shuffle)
    val_dataloader = load_data(X=X_val_scaled, y=y_val_scaled, batch_size=batch_size, shuffle=shuffle)

    kwargs = {
        'input_size': X.shape[-1],
        'hidden_size': 256 , # 64 and 128 have weird spikes at the end
        'num_layers': 16,
        'train_dataloader': train_dataloader,
        'val_dataloader': val_dataloader,
        'scaler_y': scaler_y,
        
    }
    # lr 0.1 is too high, will fluctuate at the end 
    # FIXME: Edit n_epoch
    model = custom_model(type='RNN', lr=0.0001, n_epoch=100, device=DEVICE, **kwargs) # It's wrong to write kwargs=kwargs
    model.fit()
    model.plot_loss()

    # model = custom_model(type='LSTM', lr=0.05, n_epoch=100, device=device, **kwargs) # It's wrong to write kwargs=kwargs
    # model.fit()
    # model.plot_loss()

# Use the last model with all the training and validation data
test_model = model
X_test, y_test = X.iloc[split_idx:], y.iloc[split_idx:]



# Test another RNN

https://github.com/Daammon/Stock-Prediction-with-RNN-in-Pytorch/blob/master/Rnn0.ipynb

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
from pathlib import Path

X = df2.drop(columns=['Future Close'])
y = df2['Future Close']

# Define the LSTM-based neural network model
class Model1(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers): # FIXME: input size
        super(Model1, self).__init__()
        # LSTM layer with specified input_size, hidden_size, and num_layers, batch_first ensures
        # that the input and output tensors are provided as (batch, seq, feature)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        # Output layer that maps the output of the LSTM to the desired output size
        self.out = nn.Linear(hidden_size, 1)
        # Batch normalization applied to outputs across the batch dimension
        self.bn = nn.BatchNorm1d(hidden_size)

    def forward(self, x, h_state, c_state):
        # Forward pass through LSTM
        r_out, (h_state, c_state) = self.lstm(x, (h_state, c_state))
        # Apply batch normalization to the outputs at each time step
        # r_out = self.bn(r_out)
        
        # # Collect the predictions for each time step
        # outs = []
        # for time_step in range(r_out.size(1)):
        #     outs.append(self.out(r_out[:, time_step, :]))
        
        # # Stack predictions into a tensor
        # return torch.stack(outs, dim=1), (h_state, c_state)
        return self.out(r_out), (h_state, c_state)

# Dataset class that handles loading of data
class StockDataset(Dataset):
    def __init__(self, X_dataset, Y_dataset):
        # Convert the input datasets to tensors
        self.X_dataset = torch.from_numpy(X_dataset).float()
        self.Y_dataset = torch.from_numpy(Y_dataset).float()
        
    def __len__(self):
        # Return the number of samples in the dataset
        return self.X_dataset.size(0)

    def __getitem__(self, idx):
        # Retrieve the sequence and the target value at the given index
        sequence = self.X_dataset[idx, :, :]
        target = self.Y_dataset[idx]

        # print(sequence.shape, target.shape)
        return {'Sequence': sequence, 'Target': target}



# Set hyperparameters
batch_size = 64
hidden_size = 20
num_layers = 4

# Initialize the neural network, transfer it to GPU if available
net = Model1(input_size=X.shape[-1], hidden_size=hidden_size, num_layers=num_layers).to(DEVICE)

# Initialize the hidden and cell states of LSTM
h_state = torch.zeros(num_layers, batch_size, hidden_size).to(DEVICE)
c_state = torch.zeros(num_layers, batch_size, hidden_size).to(DEVICE)

# Define a loss function and the optimizer 
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.00005)


train_dataset = StockDataset(load_sequence(X, 20), y.to_numpy()) # FIXME: Might have diff length
# train_dataset = load_data(load_sequence(X, 20), y, 64, False)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=False, drop_last=True)


# Training loop
for epoch in range(100):
    running_loss = 0.0
    prediction_plot = []

    net.train()
    # Iterate over the DataLoader for training data
    for i, data in enumerate(train_dataloader):
        inputs, labels = data['Sequence'], data['Target']
        
        # Wrap inputs and labels in Variables, transfer them to GPU if available
        x = Variable(inputs).to(DEVICE)
        y = Variable(labels).to(DEVICE)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass, backward pass, and optimize
        prediction, (h_state, c_state) = net(x, h_state, c_state)
        # Get the predictions for the last time step
        # print(prediction)
        prediction = prediction[:, 19, :].squeeze() # FIXME: hardcoded
        # Detach states from the graph to prevent backpropagation through the entire sequence
        h_state = Variable(h_state.data)
        c_state = Variable(c_state.data)
        loss = criterion(prediction, y)

        loss.backward()
        for name, param in net.named_parameters():
            print(f"requires_grad: {param.requires_grad}")
            if param.grad is not None:
                print(f"Layer: {name} | Max Gradient: {param.grad.data.abs().max()}")
            else:
                print(f"Gradient is None")
        
        optimizer.step()
        
        # Print loss statistics
        running_loss += loss.item()
        if i % 2 == 1:
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 2:.7f}')
            running_loss = 0.0
        
        # Store predictions for plotting
        prediction_plot.extend(prediction.cpu().detach().numpy())

        # Detach states after updating to cut off the graph for the next batch
        # h_state = h_state.detach()
        # c_state = c_state.detach()

print('Finished Training')

In [ ]:
plt.plot(prediction_plot)

# Prediction
- Add SHAP values

In [ ]:
# Parameters are similar to that in training
_, X_test_scaled = scale(X_train=X_train, X_val=X_test, type=scaler_type)
test_dataloader = load_data(X=X_test, y=y_test, batch_size=batch_size, shuffle=shuffle)
pred_dict, avg_test_loss = test_model.evaluate(test_dataloader=test_dataloader)


In [ ]:
pred_df = pd.DataFrame(pred_dict)
pred_list = pred_dict['predicted']
actual_list = pred_dict['actual']

from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(y_true=actual_list, y_pred=pred_list)
print(f"RMSE: {rmse:.5f}, average test loss: {avg_test_loss:.5f} should be similar") 


plt.figure(figsize=(10, 6))
x = range(len(pred_list))

plt.plot(x, pred_list, label='Predicted')
plt.plot(x, actual_list, label='Actual')
plt.title('Actual vs Predicted for Test Data')
plt.legend()
plt.show()

In [ ]:
test_model.model.eval()
X_train_tensor = torch.tensor(X_train_scaled[:1000], dtype=torch.float32).to(DEVICE) # FIXME: Edit size
X_test_tensor = torch.tensor(X_test_scaled[:5], dtype=torch.float32).to(DEVICE)
# Need to use DeepExplainer for PyTorch, instead of Explainer
# Analyse how each feature contributes to the prediction
deepExplainer = shap.DeepExplainer(model=test_model.model, data=X_train_tensor)
# Approximate shap values for the prediction
shap_values = deepExplainer.shap_values(X_test_tensor)

In [ ]:
shap_values_squeezed = np.squeeze(shap_values, axis=2)
shap_values_squeezed.shape
shap.summary_plot(shap_values_squeezed, X_test_tensor)

In [ ]:
X_test_numpy = X_test_tensor.cpu().numpy()
X_test_numpy.shape

In [ ]:
shap.initjs()
# deepExplainer.expected_value = deepExplainer.expected_value[0]
shap.plots.force(deepExplainer.expected_value, shap_values_squeezed, X_test_numpy, matplotlib=False)

Afterwards for really testing against test

In [ ]:
#
# X_train, X_val, y_train, y_val = train_test_split(X.iloc[:split_idx], y.iloc[:split_idx], shuffle=False, test_size=0.2)